Restrucutred version of the same named script in the `scripts` directory. Used to understand the code better and analysze whats going on.

In [1]:
import copy
from typing import Tuple

# isaac gym has to be imported before torch
from isaac_evaluation.grasp_quality_evaluation import GraspSuccessEvaluator
import trimesh

import scipy.spatial.transform
import numpy as np
from se3dif.datasets import AcronymGraspsDirectory
from se3dif.models.loader import load_model
from se3dif.samplers import ApproximatedGrasp_AnnealedLD, Grasp_AnnealedLD
from se3dif.utils import to_numpy, to_torch
from se3dif.visualization import grasp_visualization

import torch

Importing module 'gym_37' (/home/moritz/Documents/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/moritz/Documents/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json


/home/moritz/miniconda3/envs/alr/envs/se3dif_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version 1.13.1+cu116
Device count 1
/home/moritz/Documents/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/moritz/.cache/torch_extensions/py37_cu116 as PyTorch extensions root...
Emitting ninja build file /home/moritz/.cache/torch_extensions/py37_cu116/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


In [2]:
N_GRASPS = 10               # argument values from README: 10,10,10. default in argparse: 200
OBJ_ID = 0                  # argument values from README: 0,10,12.  default in argparse: 0
OBJ_CLASS = "ScrewDriver"   # argument values from README: 'ScrewDriver','grasp_dif_mugs','Mug'. default in argparse: "Laptop"
N_ENVS = 30                 # hardcoded in the code with default value 30 in __main__
DEVICE = "cuda:0"           # argumant values from README: n.a., n.a., n.a. default in argparse: "cuda:0"
EVAL_SIM = False            # argument values from README: n.a., n.a., n.a. default in argparse: False
MODEL = "grasp_dif_multi"   # argument values from README: n.a., 'grasp_dif_mugs', n.a. default in argparse: 'grasp_dif_multi'
BATCH = 10                  # hardcoded in code with default value 10 in get_approximated_grasp_diffusion_field

In [3]:
def get_object(
    obj_id: int, obj_class: str
) -> Tuple["NpArray[3, N]", trimesh.Trimesh, "NpArray[4, 4]", "NpArray[4, 4]"]:
    """Load the mesh of the specified object and sample a pointcloud from it.
    The pointcloud is rotated randomly, scaled by 8 and made zero mean.

    Args:
        obj_id (int): The id of the object to load.
        obj_class (str): The class of the object to load.

    Returns:
        Tuple["NpArray[3, N]", trimesh.Trimesh, "NpArray[4, 4]", "NpArray[4]"]: The pointcloud, 
            the mesh, the homogenaous transformation matrix of the rotation and the homogeneous
            transformation matrix of the translation.
    """
    
    # get mesh of object and sample pointcloud
    acronym_grasps = AcronymGraspsDirectory(data_type=obj_class)
    mesh = acronym_grasps.avail_obj[obj_id].load_mesh()
    pointcloud = mesh.sample(1000)

    # get a random rotation
    H_rot = np.eye(4)
    H_rot[:3, :3] = scipy.spatial.transform.Rotation.random().as_matrix()

    # TODO refactor this

    # apply the rotation to the pointcloud, scale the pointcloud by 8 and make it zero mean
    pointcloud = np.einsum("mn,bn->bm", H_rot[:3,:3], pointcloud)
    pointcloud *= 8.0
    pointcloud_mean = np.mean(pointcloud, 0)
    pointcloud += -pointcloud_mean

    # apply the rotation to the mesh, scale it by 8 and make it zero mean
    mesh.apply_transform(H_rot)
    mesh.apply_scale(8.0)
    H_trans = np.eye(4)
    H_trans[:3, -1] = -pointcloud_mean
    mesh.apply_transform(H_trans)

    return pointcloud, mesh, H_rot, H_trans

In [4]:
# print(
#     "P: ",
#     pointcloud.min(0),
#     pointcloud.max(0),
#     pointcloud.shape,
#     pointcloud.dtype,
#     pointcloud.mean(0),
# )
# print(
#     "mesh: ",
#     type(mesh),
#     mesh.vertices.min(0),
#     mesh.vertices.max(0),
#     mesh.vertices.shape,
#     mesh.vertices.dtype,
#     mesh.vertices.mean(0),
# )
# print("translational shift", translational_shift)
# print("rotational quat", rot_quat)

NameError: name 'pointcloud' is not defined

In [ ]:
# generator, model = get_approximated_grasp_diffusion_field(P, args, device)
# def get_approximated_grasp_diffusion_field(p, args, device='cpu'):


model = load_model({"device": DEVICE, "pretrained_model": MODEL})

def get_grasp_generator(pointcloud, model):
    ## Load model

    model.set_latent(to_torch(pointcloud[None, ...], DEVICE), batch=BATCH)

    ########### 2. SET SAMPLING METHOD #############
    generator = Grasp_AnnealedLD(
        model, batch=BATCH, T=70, T_fit=50, k_steps=2, device=DEVICE
    )

In [ ]:
print("generator: ", type(generator))
print("model: ", type(MODEL))

In [ ]:
H = generator.sample()

H_grasp = copy.deepcopy(H)
# counteract the translational shift of the pointcloud (as the spawned model in simulation will still have it)
H_grasp[:, :3, -1] = (H_grasp[:, :3, -1] - torch.as_tensor(translational_shift[:3,-1],device=DEVICE)).float()
H[..., :3, -1] *=1/8.
H_grasp[..., :3, -1] *=1/8.

## Visualize results ##

vis_H = H.squeeze()
pointcloud *=1/8
mesh = mesh.apply_scale(1/8)
scene = grasp_visualization.visualize_grasps(to_numpy(H), p_cloud=pointcloud, mesh=mesh, show=False)

In [ ]:
scene.show()

In [ ]:
# if (EVAL_SIM):
#     ## Evaluate Grasps in Simulation##
#     num_eval_envs = 10
#     evaluator = GraspSuccessEvaluator(OBJ_CLASS, n_envs=num_eval_envs, idxs=[args.obj_id] * num_eval_envs, viewer=True, device=DEVICE, \
#                                         rotations=[rot_quad]*num_eval_envs, enable_rel_trafo=False)
#     succes_rate = evaluator.eval_set_of_grasps(H_grasp)
#     print('Success cases : {}'.format(succes_rate))